# Linear Regression
The goal of this notebook is to implement linear regression from scratch as practice to refresh my knowledge of the process.
The goal of using Linear Regression is to find parameters $\beta$ such that $X \beta = y$ where $X$ is our matrix of features and $y$ is our target variable. 

The example dataset for this notebook is the boston housing dataset from sklearn


In [20]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston

In [21]:
boston = load_boston()

In [35]:
boston_X = boston['data']
boston_X_with_intercept = np.concatenate([boston_X,np.ones((boston_X.shape[0],1))],axis= 1)
boston_y = boston['target']

The first way I will implement linear regression is using ordinary least squares and minimizing the sum of squared residuals. This can be solved with the formula: 
$$ \hat{\beta} = (X^\top X)^{-1} X^\top y $$
Where $\hat{\beta}$ is our estimated parameters. This assumes that the columns of $X$ are linearly independent and that there exists a linear relationship between $X$ and $y$

In [2]:
def basic_linear_regression(X,y):
    '''
    Takes in a feature matrix X and a target matrix y and returns beta_hat which is the 
    estimated parameters such that X * beta_hat = y_hat 
    This function will assume that the first column of X is a column of ones that will
    will be used to fit the intercept
    '''
    #the inverse of X transpose times X 
    XTX_inv =  np.linalg.inv(np.dot(X.T,X))
    #times X transpose times y
    beta_hat = np.dot(np.dot(XTX_inv, X.T),y)
    return beta_hat

This function will implement the cost function, the root mean square error
$$ L(\hat{y},y) = \sqrt{\frac{1}{n}\sum_{i = 0}^n (y_i - \hat{y_i})^2} $$
This will be the cost function that I will use to evaluate my implementations. 

In [62]:
# Implement the cost function
def cost_function(y, y_hat):
    mean_squares = np.mean((np.array(y) - np.array(y_hat))**2)
    RMSE = np.sqrt(mean_squares)
    return RMSE

Now I'll use my implementation and see the RMSE, and compare it to scikit-learns implementation, both on the training set and a holdout set.

In [46]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [50]:
#create train and test sets 
X_train, X_test, y_train, y_test = train_test_split(boston_X_with_intercept,boston_y,test_size = .2)

In [55]:
# my implementation, with the RMSE on both the training and test sets

beta_hat = basic_linear_regression(X_train,y_train)

y_train_hat = X_train.dot(beta_hat)
RMSE = cost_function(y_train,y_train_hat)
print("training error: ",RMSE)

y_test_hat = X_test.dot(beta_hat)
RMSE_test = cost_function(y_test,y_test_hat)
print("test error: ", RMSE_test)

training error:  4.74482379421
test error:  4.47084812241


In [57]:
#comparing to sklearn's implementation
lr = LinearRegression()
lr.fit(X=boston_X_with_intercept, y = boston_y)

y_train_hat = lr.predict(X_train)
RMSE = cost_function(y_train,y_train_hat)
print("training error: ",RMSE)

y_test_hat = lr.predict(X_test)
RMSE_test = cost_function(y_test,y_test_hat)
print("test error: ", RMSE_test)

training error:  4.75594217932
test error:  4.36362631944


In [61]:
boston_y.std()

9.1880115452782025

Not bad, sklearns implementation did slightly better on the test set, but they have compareable RMSE, at about half of the standard deviation of the target set. 

In [17]:
def find_gradient(X,y,y_hat):
    return np.dot(X.T, y - y_hat)

In [19]:
def iteration(X,y,beta, learning_rate):
    '''
    '''
    y_hat = X.dot(beta)
    gradient = np.dot(X.T, y - y_hat)
    beta = beta - learning_rate*gradient
    return beta